In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2

labels = os.listdir('openClosedFaces')
path = os.path.join('openClosedFaces', 'OpenFace')
image = cv2.imread(os.path.join(path, 'Abdel_Madi_Shabneh_0001.jpg'), cv2.IMREAD_COLOR)
image.shape

(100, 100, 3)

In [8]:
from tqdm import tqdm
def get_yawn_data(direc="train"):
    yaw_no = []
    IMG_SIZE = 145
    categories = ["yawn", "no_yawn"]
    for category in categories:
        path_link = os.path.join(direc, category)
        class_num1 = categories.index(category)
        print(class_num1)
        for image in tqdm(os.listdir(path_link)):
            image_array = cv2.imread(os.path.join(path_link, image), cv2.IMREAD_COLOR)
            face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
            faces = face_cascade.detectMultiScale(image_array, 1.3, 5)
            for (x, y, w, h) in faces:
                img = cv2.rectangle(image_array, (x, y), (x+w, y+h), (0, 255, 0), 2)
                roi_color = img[y:y+h, x:x+w]
                resized_array = cv2.resize(roi_color, (IMG_SIZE, IMG_SIZE))
                yaw_no.append([resized_array, class_num1])
    return yaw_no

def get_eye_data(dir_path="train"):
    labels = ['Closed', 'Open']
    IMG_SIZE = 145
    data = []
    for label in labels:
        path = os.path.join(dir_path, label)
        class_num = labels.index(label)
        class_num +=2
        print(class_num)
        for img in tqdm(os.listdir(path)):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
                resized_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                data.append([resized_array, class_num])
            except Exception as e:
                print(e)
    return data

def get_open_close_face_data(dir_path="openClosedFaces"):
    labels = ['ClosedFace', 'OpenFace']
    data = []
    for i,label in enumerate(labels):
        path = os.path.join(dir_path, label)
        for img in tqdm(os.listdir(path)):
            try:
                img_array = cv2.imread(os.path.join(path, img))# , cv2.IMREAD_COLOR)
                if img_array is not None:
                    data.append([img_array, i])
            except Exception as e:
                print(e)
    return np.array(data)

data = get_open_close_face_data()
X,y = list(zip(*data))

100%|██████████| 1232/1232 [00:00<00:00, 2372.66it/s]


In [9]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

X = np.array(X)
y = LabelBinarizer().fit_transform(y)
y = np.array(y)

test_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

In [10]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

train_generator = ImageDataGenerator(rescale=1/255, zoom_range=0.2, horizontal_flip=True, rotation_range=30)
test_generator = ImageDataGenerator(rescale=1/255)

train_generator = train_generator.flow(np.array(X_train), y_train, shuffle=False)
test_generator = test_generator.flow(np.array(X_test), y_test, shuffle=False)

In [32]:
a,b = next(train_generator)
a.shape, b.shape

((32, 100, 100, 3), (32, 1))

In [64]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation="relu", input_shape=X_train.shape[1:]))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(128, (3, 3), activation="relu"))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(256, (3, 3), activation="relu"))
model.add(MaxPooling2D(2, 2))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer=tf.keras.optimizers.Adam(clipnorm=1.0))
model.summary()

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_68 (Conv2D)           (None, 98, 98, 32)        896       
_________________________________________________________________
max_pooling2d_68 (MaxPooling (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_69 (Conv2D)           (None, 47, 47, 64)        18496     
_________________________________________________________________
max_pooling2d_69 (MaxPooling (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_70 (Conv2D)           (None, 21, 21, 128)       73856     
_________________________________________________________________
max_pooling2d_70 (MaxPooling (None, 10, 10, 128)       0         
_________________________________________________________________
conv2d_71 (Conv2D)           (None, 8, 8, 256)       

In [65]:
history = model.fit(train_generator, epochs=50, validation_data=test_generator, shuffle=True, validation_steps=len(test_generator))

Epoch 1/50
53/53 [==============================] - 3s 56ms/step - loss: 0.6768 - accuracy: 0.5761 - val_loss: 0.6777 - val_accuracy: 0.5983
Epoch 2/50
53/53 [==============================] - 3s 52ms/step - loss: 0.6348 - accuracy: 0.6504 - val_loss: 0.6179 - val_accuracy: 0.6740
Epoch 3/50
53/53 [==============================] - 3s 53ms/step - loss: 0.6226 - accuracy: 0.6621 - val_loss: 0.5825 - val_accuracy: 0.6878
Epoch 4/50
53/53 [==============================] - 3s 52ms/step - loss: 0.5900 - accuracy: 0.6822 - val_loss: 0.5321 - val_accuracy: 0.7400
Epoch 5/50
53/53 [==============================] - 3s 53ms/step - loss: 0.5579 - accuracy: 0.7193 - val_loss: 0.5513 - val_accuracy: 0.7221
Epoch 6/50
53/53 [==============================] - 3s 53ms/step - loss: 0.5149 - accuracy: 0.7559 - val_loss: 0.4595 - val_accuracy: 0.7703
Epoch 7/50
53/53 [==============================] - 3s 52ms/step - loss: 0.4890 - accuracy: 0.7712 - val_loss: 0.3820 - val_accuracy: 0.8198
Epoch 8/50
53

In [66]:
labels_new = ['ClosedFace', 'OpenFace']
prediction = model.predict_classes(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, prediction, target_names=labels_new))

              precision    recall  f1-score   support

  ClosedFace       0.93      0.97      0.95       376
    OpenFace       0.96      0.92      0.94       351

    accuracy                           0.94       727
   macro avg       0.95      0.94      0.94       727
weighted avg       0.95      0.94      0.94       727



In [11]:
model.save("eyesOpenClose.model")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: drowsiness.model/assets


In [18]:
model.save("eyesOpenClose.h5")